## Assignment 4
#### Group 1: Tara Bode, Hankun Li

Part A: Text-based industry classification
Cluster the business sections extracted in assignment 3 into 50 clusters (using the cosine similarity) to group firms with similar business descriptions together.

Submit your code and the final output csv file, with CIK, conformed end of period, and the cluster number for each business section file.

Note: it is likely you will need to tokenize/word count each business section file before reading the next one. (If you try to read all business sections before doing any further processing you will probably run out of memory).

In [9]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

You need to simply take the.txt files from HiperGator and use the path (r'/blue/acg7849/share/BS/') 

so we can grab the length of each file for the csv file.

Also, we need to use the for loop to cluster the business section files and mimic the class code from yesterday to grab the top 20 common words.

# all functions

In [29]:
def getHeaderVariables(filing):
    CIK = filing[ filing.find('CENTRAL INDEX KEY:') + 18  : filing.find('STANDARD') ]
    CIK = ''.join(CIK.split())
    #
    CCN = filing[ filing.find('COMPANY CONFORMED NAME:') + 23 : filing.find('CENTRAL INDEX KEY') ]
    CCN = CCN.rstrip().lstrip()
    #
    CST = filing[ filing.find('CONFORMED SUBMISSION TYPE:') + 26 : filing.find('PUBLIC DOCUMENT COUNT') ]
    CST = ''.join(CST.split())
    #
    CPR = filing[ filing.find('CONFORMED PERIOD OF REPORT:') + 27 : filing.find('FILED AS OF DATE') ]
    CPR = ''.join(CPR.split())
    #
    Length = 'skip'
    
    sec_info = '{} | {} | {} | {} | {}\n'.format(CIK, CCN, CST, CPR, Length)
    return (sec_info)


In [11]:
def cleanHTML(raw_html):
    cleanr = re.compile('<.*?>', flags=re.DOTALL)
    cleantext = re.sub(cleanr, '', raw_html)
    return replace_entities(cleantext)

In [12]:
def getBusinessSection(filing):
    # USE REGEXR
    BDS = filing[ filing.find('Item 1. ') : filing.find('Item 1A. ') ]
    #print('length_business_section',len(BDS))
    #print(BDS)
    return BDS
    #

# main loop

csv file, with CIK, conformed end of period, and file name (business section file) and the cluster number for each business section file.

In [10]:
#mydirectory = '/blue/acg7849/share/10Ks'
mydirectory = '/blue/acg7849/share/BS/item1'
file_list = glob.glob(mydirectory + '/*.txt')
# print(file_list) # success!

In [9]:
obj = pd.DataFrame(file_list)
obj.to_csv('file_list.csv', index = True) # csv preapared to use

In [24]:
badFiles_raw = []
#header vars
with open (r'/blue/acg7849/hli1/SEC_Information.txt', 'w') as f:
    f.write('CIK|coname|formtype|date|filename|length|cluster\n')

In [34]:
for i in file_list[0:5]:
    print (i)
    # read the file
    with open(i) as f:
        filing = f.read()
        
    # grab the length of the file
    print (len(filing))

    # Ignore business section files with less than 1000 characters.
    if len(filing) < 50000: # test. should be 1000 here
        print ('bad')
        badFiles_raw.append (i)
    # should end this file and return to next one
    
    # business info
    # get header variables
    #first_section = filing[0: filing.find('</DOCUMENT')]
   # headervars = str(getHeaderVariables(first_section))
    #print(headervars)
    
    #need to input FileName
    shortName = i[i.rfind("/")+1:]
    print(shortName)
    # tokenize
    #word_tokenize (filing[0:10]) # bit different from split - include punctuations
#print(badFiles_raw)

/blue/acg7849/share/BS/item1/267762.txt
66145
267762.txt
/blue/acg7849/share/BS/item1/280603.txt
57117
280603.txt
/blue/acg7849/share/BS/item1/267435.txt
28236
bad
267435.txt
/blue/acg7849/share/BS/item1/275323.txt
38468
bad
275323.txt
/blue/acg7849/share/BS/item1/277539.txt
36429
bad
277539.txt


In [17]:
print(badFiles_raw)

['/blue/acg7849/share/BS/item1/267435.txt']


In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [2]:

# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [3]:

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [4]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in synopses:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)